In [47]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo

!pip install gdown

!gdown 1hKQDX7SZw5AzysFh32gr0qxTOHLzAQIC

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1hKQDX7SZw5AzysFh32gr0qxTOHLzAQIC
To: /kaggle/working/SemEval2025_task11_processed.zip
100%|███████████████████████████████████████| 2.99M/2.99M [00:00<00:00, 174MB/s]


In [48]:
!rm -rf processed_data

!unzip -q SemEval2025_task11_processed.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [50]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments

from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit



In [51]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [52]:
kind="train"

task="track_a"

langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]

# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [53]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"

    if not os.path.isfile(processed_path):

      print(processed_path)

      continue

    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




processed_data/train/track_a/sum.csv


In [54]:
train_data=all_csv.drop(columns={"unnamed: 0"})

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,clean_message,surprise
0,afr_train_track_a_00001,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,afr_train_track_a_00002,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,afr_train_track_a_00003,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,afr_train_track_a_00004,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,afr_train_track_a_00005,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,tir_train_track_a_03678,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,tir_train_track_a_03679,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,tir_train_track_a_03680,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [55]:
train_data = train_data[

    train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,clean_message,surprise
0,0,afr_train_track_a_00001,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,1,afr_train_track_a_00002,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,2,afr_train_track_a_00003,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,3,afr_train_track_a_00004,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,4,afr_train_track_a_00005,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...,...
25557,25557,tir_train_track_a_03677,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,25558,tir_train_track_a_03678,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,25559,tir_train_track_a_03679,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,25560,tir_train_track_a_03680,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [56]:
#update label column name

label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']



y = train_data[label_columns].values



mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=1)

train_indices, valid_indices = next(mskf.split(train_data, y))



train = train_data.iloc[train_indices].reset_index(drop=True)

valid = train_data.iloc[valid_indices].reset_index(drop=True)



x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()



y_train_df = pd.DataFrame(y_train, columns=label_columns)



label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']

print(label_distribution)

          count_0  count_1
anger       16358     4092
fear        17983     2467
joy         16022     4428
sadness     16960     3490
surprise    18594     1856


In [57]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [58]:
from torch.nn import BCEWithLogitsLoss
import torch.nn as nn
import torch
from transformers import AutoModel

class EnhancedMultilingualClassifier(nn.Module):
    def __init__(self, model_name, num_labels, pos_weight=None):
        super(EnhancedMultilingualClassifier, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_name)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.hidden_size = self.base_model.config.hidden_size
        self.bilstm = nn.LSTM(
            input_size=self.hidden_size,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        self.attention = nn.Linear(self.hidden_size * 2, 1)
        self.dropout = nn.Dropout(0.3)
        self.layer_norm = nn.LayerNorm(self.hidden_size * 2)
        self.hidden_layer1 = nn.Linear(self.hidden_size * 2, 1024)
        self.hidden_layer2 = nn.Linear(1024, 512)
        self.hidden_layer3 = nn.Linear(512, num_labels)
        self.activation = nn.GELU()
        self.loss_fn = BCEWithLogitsLoss(pos_weight=pos_weight)

    def forward(self, input_ids, attention_mask, labels=None):
        base_output = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = base_output.last_hidden_state
        lstm_output, _ = self.bilstm(sequence_output)
        lstm_output = self.layer_norm(lstm_output)
        attention_scores = torch.softmax(self.attention(lstm_output), dim=1)
        context_vector = torch.sum(attention_scores * lstm_output, dim=1)
        x = self.dropout(context_vector)
        x = self.activation(self.hidden_layer1(x))
        x = self.dropout(x)
        x = self.activation(self.hidden_layer2(x))
        x = self.dropout(x)
        logits = self.hidden_layer3(x)
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return (loss, logits) if loss is not None else logits


In [59]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [60]:
def build_dataset(x_train, y_train, x_valid, y_valid, tokenizer, max_length):
    train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=max_length)
    valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=max_length)
    train_dataset = Dataset(train_encodings, y_train)
    valid_dataset = Dataset(valid_encodings, y_valid)
    return train_dataset, valid_dataset


def compute_metrics(pred):
    predictions = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    probabilities = torch.sigmoid(torch.tensor(predictions))
    predicted_classes = (probabilities >= 0.5).int()
    precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average="weighted")
    accuracy = accuracy_score(y_valid, predicted_classes)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}


In [61]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = len(label_columns)
MAX_LENGTH = 256
model = EnhancedMultilingualClassifier(model_name=model_name, num_labels=num_labels)
model.to(device)

EnhancedMultilingualClassifier(
  (base_model): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=76

In [62]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=3e-5,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=1.0,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [63]:
train_dataset, valid_dataset = build_dataset(x_train, y_train, x_valid, y_valid, tokenizer, MAX_LENGTH)

In [64]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/tmp/ipykernel_30/3211253830.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [65]:
import shutil
shutil.rmtree(training_args.output_dir)

In [66]:
checkpoints = sorted(glob.glob(os.path.join(training_args.output_dir, "checkpoint-*")), key=os.path.getmtime)
resume_checkpoint = checkpoints[-1] if checkpoints else None

trainer.train(resume_from_checkpoint=resume_checkpoint)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
0,0.389500,0.373170,0.388889,0.494775,0.150943,0.214875
1,0.345700,0.327032,0.497848,0.652948,0.416565,0.488881
2,0.290400,0.321271,0.510368,0.659730,0.463612,0.523915
3,0.250900,0.304793,0.542254,0.673511,0.518990,0.577034
4,0.213400,0.296711,0.559859,0.688688,0.544964,0.606015
5,0.187400,0.313018,0.556534,0.666830,0.565548,0.610942
6,0.161300,0.331810,0.549883,0.638362,0.606714,0.620918
7,0.147000,0.347526,0.558685,0.664648,0.562607,0.606860


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2556, training_loss=0.2624009388480015, metrics={'train_runtime': 5126.1943, 'train_samples_per_second': 39.893, 'train_steps_per_second': 0.622, 'total_flos': 0.0, 'train_loss': 0.2624009388480015, 'epoch': 7.999218139171227})

In [67]:
results = trainer.evaluate()
print(f"Validation results: {results}")

Validation results: {'eval_loss': 0.2967108190059662, 'eval_accuracy': 0.5598591549295775, 'eval_precision': 0.6886881989081763, 'eval_recall': 0.5449644694927713, 'eval_f1_score': 0.6060148904782062, 'eval_runtime': 45.6393, 'eval_samples_per_second': 112.009, 'eval_steps_per_second': 3.506, 'epoch': 7.999218139171227}


In [68]:
trainer.save_model("./output/final_model")
tokenizer.save_pretrained("./output/final_model")

('./output/final_model/tokenizer_config.json',
 './output/final_model/special_tokens_map.json',
 './output/final_model/sentencepiece.bpe.model',
 './output/final_model/added_tokens.json',
 './output/final_model/tokenizer.json')